In [ ]:
import numpy as np
from scipy.optimize import minimize_scalar, minimize
from scipy.optimize import NonlinearConstraint

In [ ]:
!pip install pulp

     |████████████████████████████████| 40.6MB 99kB/s 


In [ ]:
# Importing data analytics related libraries (DataFrame and Visualization)
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style

# Adjusting the size of matplotlib
import matplotlib as mpl
mpl.rc('figure', figsize=(8, 7))
mpl.__version__

# Adjusting the style of matplotlib
style.use('ggplot')

# Magic to tell python notebook we want matplotlib charts included
%matplotlib inline

# importing pulp as an optimization solver
from pulp import *
import re 

In [ ]:
import xlrd
import numpy as np

df = pd.read_excel('Trade_Spend_New.xlsm', sheet_name='Working')

new_header = df.iloc[1] #grab the first row for the header
df = df[2:] #take the data less the header row
df.columns = new_header #set the header row as the df header


df.head(5)

1,Retailer,Item Code,Division Name,Business Unit,Item Description,Location Code,Location Name,City,State,Zip Code,Date,Week,Sum of Actual ST,Sum of Forecast ST,Sum of Forecast SI,Sum of Opening OH,Sum of Closing OH,PED,ASP,% Discount,% Change in SI,Projected Forecast ST,Projected Forecast SI,Project Opening OH,Project Closing OH,Project ST (Act),Trade Spend,Forecasted ST/AC,Cost,GP%,GP,Rev,GP Base,Rev Base,ST Rate Simu,In Stk %,In Stk %,Sim FC ST/Act ST,NaN,NaN
2,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-01 00:00:00,2020-11-01 00:00:00,74,85,0,0,459,1.13,2599.99,0.00665341,0.0232472,85,0,0,459,85,0,74,1040,0.599998,115439,192399,115439,192399,0,1,1,74,NaN,0.599998
3,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-02 00:00:00,2020-11-02 00:00:00,109,103,215,459,565,1.13,2599.99,0.00665341,0.0232472,103,215,459,565,103,0,109,1040,0.599998,170039,283399,170039,283399,0.161721,1,1,109,NaN,0.599998
4,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-03 00:00:00,2020-11-02 00:00:00,31,35,0,565,534,1.13,2599.99,0.00665341,0.0232472,35,0,565,534,35,0,31,1040,0.599998,48360,80600,48360,80600,0.0548673,1,1,31,NaN,0.599998
5,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-04 00:00:00,2020-11-02 00:00:00,110,108,0,534,424,1.13,2599.99,0.00665341,0.0232472,108,0,534,424,108,0,110,1040,0.599998,171599,285999,171599,285999,0.205993,1,1,110,NaN,0.599998
6,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-05 00:00:00,2020-11-02 00:00:00,46,49,143,424,521,1.13,2599.99,0.00665341,0.0232472,49,143,424,521,49,0,46,1040,0.599998,71760,119600,71760,119600,0.0811287,1,1,46,NaN,0.599998


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.tail()

1,Retailer,Item Code,Division Name,Business Unit,Item Description,Location Code,Location Name,City,State,Zip Code,Date,Week,Sum of Actual ST,Sum of Forecast ST,Sum of Forecast SI,Sum of Opening OH,Sum of Closing OH,PED,ASP,% Discount,% Change in SI,Projected Forecast ST,Projected Forecast SI,Project Opening OH,Project Closing OH,Project ST (Act),Trade Spend,Forecasted ST/AC,Cost,GP%,GP,Rev,GP Base,Rev Base,ST Rate Simu,In Stk %,In Stk %,Sim FC ST/Act ST,NaN,NaN
1195,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-24 00:00:00,2021-02-22 00:00:00,0,60,0,0,0,0.95,999.99,0.0203178,0.0134795,58,0,0,0,0,0,0,600,0.38755,0,0,0,0,0,0,0,0,NaN,0.399994
1196,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-25 00:00:00,2021-02-22 00:00:00,0,79,118,0,39,0.95,999.99,0.0203178,0.0134795,77,120,0,43,77,1560,79,600,0.38755,29235,75435,31599,78999,0.641667,1,1,77,NaN,0.399994
1197,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-26 00:00:00,2021-02-22 00:00:00,0,64,0,39,0,0.95,999.99,0.0203178,0.0134795,62,0,43,0,43,870,39,600,0.38755,16326,42126,15600,39000,1,0,0,43,NaN,0.399994
1198,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-27 00:00:00,2021-02-22 00:00:00,0,66,0,0,0,0.95,999.99,0.0203178,0.0134795,64,0,0,0,0,0,0,600,0.38755,0,0,0,0,0,0,0,0,NaN,0.399994
1199,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-28 00:00:00,2021-02-22 00:00:00,0,42,0,0,0,0.95,999.99,0.0203178,0.0134795,41,0,0,0,0,0,0,600,0.38755,0,0,0,0,0,0,0,0,NaN,0.399994


In [ ]:
new_header

Unnamed: 0                            Retailer
Unnamed: 1                           Item Code
Unnamed: 2                       Division Name
Unnamed: 3                       Business Unit
Unnamed: 4                    Item Description
Unnamed: 5                       Location Code
Unnamed: 6                       Location Name
Unnamed: 7                                City
Unnamed: 8                               State
Unnamed: 9                            Zip Code
Unnamed: 10                               Date
Unnamed: 11                               Week
Unnamed: 12                   Sum of Actual ST
Unnamed: 13                 Sum of Forecast ST
Unnamed: 14                 Sum of Forecast SI
Unnamed: 15                  Sum of Opening OH
Unnamed: 16                  Sum of Closing OH
Unnamed: 17                                PED
-172.0001000000002                         ASP
2771.9901                           % Discount
Unnamed: 20                     % Change in SI
93           

In [ ]:
for col in df.columns: 
  print(col)

Retailer
Item Code
Division Name
Business Unit
Item Description
Location Code
Location Name
City
State
Zip Code
Date
Week
Sum of Actual ST
Sum of Forecast ST
Sum of Forecast SI
Sum of Opening OH
Sum of Closing OH
PED
ASP
% Discount
% Change in SI
Projected Forecast ST
Projected Forecast SI
Project Opening OH
Project Closing OH
Project ST (Act)
Trade Spend
Forecasted ST/AC
Cost
GP%
GP
Rev
GP Base
Rev Base
ST Rate Simu
In Stk %
In Stk %
Sim FC ST/Act ST
nan
nan


In [ ]:
# Remove unnecessary columns
df2 = df.copy()
df2.head()

df2 = df2.loc[:, df2.columns.isin(['Retailer','Item Code', 'Division Name', 
                                    'Business Unit', 'Item Description','Location Code','Location Name', 'City', 'State',
                                    'Zip Code', 'Date', 'Week', 'Sum of Actual ST','Sum of Forecast ST', 'Sum of Forecast SI',
                                    'Sum of Opening OH', 'Sum of Closing OH','PED', 'ASP'])]



In [ ]:
df2 = df2.dropna()

In [ ]:
len(df.index)

1200

In [ ]:
len(df2.index)

1200

In [ ]:
#today = pd.to_datetime("today").normalize() #for real time
df2['Date'] = pd.to_datetime(df2['Date'])

In [ ]:
today =pd.Timestamp("2021-01-11") # for testing

In [ ]:
df2['today'] = today


In [ ]:
df2.head()

1,Retailer,Item Code,Division Name,Business Unit,Item Description,Location Code,Location Name,City,State,Zip Code,Date,Week,Sum of Actual ST,Sum of Forecast ST,Sum of Forecast SI,Sum of Opening OH,Sum of Closing OH,PED,ASP,today
0,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-01,2020-11-01 00:00:00,74,85,0,0,459,1.13,2599.99,2021-01-11
1,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-02,2020-11-02 00:00:00,109,103,215,459,565,1.13,2599.99,2021-01-11
2,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-03,2020-11-02 00:00:00,31,35,0,565,534,1.13,2599.99,2021-01-11
3,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-04,2020-11-02 00:00:00,110,108,0,534,424,1.13,2599.99,2021-01-11
4,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-05,2020-11-02 00:00:00,46,49,143,424,521,1.13,2599.99,2021-01-11


In [ ]:
df2.tail()

1,Retailer,Item Code,Division Name,Business Unit,Item Description,Location Code,Location Name,City,State,Zip Code,Date,Week,Sum of Actual ST,Sum of Forecast ST,Sum of Forecast SI,Sum of Opening OH,Sum of Closing OH,PED,ASP,today
1195,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-24,2021-02-22 00:00:00,0,60,0,0,0,0.95,999.99,2021-01-11
1196,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-25,2021-02-22 00:00:00,0,79,118,0,39,0.95,999.99,2021-01-11
1197,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-26,2021-02-22 00:00:00,0,64,0,39,0,0.95,999.99,2021-01-11
1198,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-27,2021-02-22 00:00:00,0,66,0,0,0,0.95,999.99,2021-01-11
1199,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-28,2021-02-22 00:00:00,0,42,0,0,0,0.95,999.99,2021-01-11


In [ ]:
sum_of_forecast_st =  df2['Sum of Forecast ST'].tolist()
ped =  df2['PED'].tolist()



In [ ]:
date_arr = df2['Date'].tolist()

In [ ]:
today_arr = df2['today'].tolist()

In [ ]:
sum_of_forecast_SI =  df2['Sum of Forecast SI'].tolist()

In [ ]:
sum_of_closing_oh = df2['Sum of Closing OH'].tolist()

In [ ]:
sum_of_opening_oh = df2['Sum of Opening OH'].tolist()

In [ ]:
asp = df2['ASP'].tolist()

In [ ]:
sum_of_actual_st = df2['Sum of Actual ST'].tolist()

In [ ]:
cost = df['Cost'].tolist()

In [ ]:
total_run = 120

In [ ]:
total_products = 10

In [ ]:
run_product = total_run * total_products

In [ ]:
scenario = 304

max_si = 0.4

max_discount = 0.20

gp_min = 0.50

budget = 600000

In [ ]:
changeinSI = [0.0] * 1200

In [ ]:
discount = np.zeros(20)

In [ ]:
# optimize
b = (0,1)
bnds = [b, b,b, b,b, b,b, b,b, b,b, b,b, b,b, b,b, b,b, b]
#bnds = (b,b,b,b,b,b,b,b)

In [ ]:
final_frame = df2.copy()

In [ ]:
def constraint2(x):
  print("ChangeinSI CONSTRAINT 2 ....")
  changeinSI = []
  i = 0
  while i < len(x):
    if i%2 != 0:
      changeinSI.append(x[i])
    i = i + 1

  print(changeinSI)
  ch_si = np.array(changeinSI)
  return max_si - ch_si #0.2

In [ ]:
def constraint1(x):
  print("DICSOUNT CONSTRAINT 1 ....")
  discount = []
  i = 0
  while i < len(x):
    if i%2 == 0:
      discount.append(x[i])
    i = i + 1

  print(discount)
  disc = np.array(discount)
  return max_discount - disc #0.2

In [ ]:
def constraint3(x): #change constraint code, directly put X values and constraint the trade spend
  i = 0
  j=0
  sum = 0
  projected_forecast_st = copy.deepcopy(sum_of_forecast_st)
  projected_forecast_SI = copy.deepcopy(sum_of_forecast_SI)
  project_closing_oh = copy.deepcopy(sum_of_closing_oh)
  project_opening_oh = copy.deepcopy(sum_of_opening_oh)
  project_st_act = copy.deepcopy(sum_of_forecast_st)
  project_st_act_minor = copy.deepcopy(sum_of_forecast_st)

  trade_spend = copy.deepcopy(sum_of_forecast_st) #trade spend
  trade_spend_minor = copy.deepcopy(sum_of_forecast_st)

  forecasted_st_ac = copy.deepcopy(sum_of_actual_st) #

  GP_base = copy.deepcopy(sum_of_actual_st) #

  sim_fc_st_act_st = copy.deepcopy(sum_of_actual_st) #

  GP = copy.deepcopy(sum_of_actual_st)

  rev_base = copy.deepcopy(sum_of_actual_st)

  rev = copy.deepcopy(sum_of_actual_st)

  k = 0
  q = 0
  while i < total_products:
    #j = 0
    q = 0
    while q < total_run:
      if (date_arr[j] < today_arr[j]):
        projected_forecast_st[j] = sum_of_forecast_st[j]
        projected_forecast_SI[j] = sum_of_forecast_SI[j]
        project_closing_oh[j] = sum_of_closing_oh[j]
        project_opening_oh[j] = sum_of_opening_oh[j]
        project_st_act[j] = sum_of_forecast_st[j]

        trade_spend[j] = 0

        forecasted_st_ac[j] = sum_of_actual_st[j]

        sim_fc_st_act_st[j] = sum_of_actual_st[j]
        
      else:
        projected_forecast_st[j] = sum_of_forecast_st[j] * (1 + x[k]) * ped[j]
        projected_forecast_SI[j] = sum_of_forecast_SI[j] * (1 + x[k+1])
        if (j>0):
          if ((project_closing_oh[j-1] + projected_forecast_SI[j] - projected_forecast_st[j])>0):
            project_closing_oh[j] = project_closing_oh[j-1] + projected_forecast_SI[j] - projected_forecast_st[j]
          else:
            project_closing_oh[j] = 0
        else:
          project_closing_oh[j] = sum_of_closing_oh[j]
        if (j>0):
          project_opening_oh[j] = project_closing_oh[j-1]
        else:
          project_opening_oh[j] = sum_of_opening_oh[j]

        project_st_act_minor[j] = project_opening_oh[j] + projected_forecast_SI[j]
        project_st_act[j] = min(project_st_act_minor[j],projected_forecast_st[j])

        trade_spend_minor[j] = project_st_act[j] * asp[j] * x[k]
        trade_spend[j] = round(trade_spend_minor[j],-1)


        forecasted_st_ac[j] = min((sum_of_forecast_SI[j] + sum_of_opening_oh[j]), sum_of_forecast_st[j])

        sim_fc_st_act_st[j] = round(min((projected_forecast_SI[j] + project_opening_oh[j]), projected_forecast_st[j]))


      GP_base[j] = round( ( asp[j] - cost[j] ) * ( forecasted_st_ac[j]) )

      if (date_arr[j] < today_arr[j]):
        GP[j] = GP_base[j]
      else: 
        GP[j] = round( (( asp[j] * (1 - x[k]) - cost[j] ) * ( sim_fc_st_act_st[j] )), - 1)


      rev_base[j] = round( ( asp[j] * forecasted_st_ac[j] ) )


      if (date_arr[j] < today_arr[j]):
        rev[j] = rev_base[j]
      else: 
        rev[j] = round( ( ( 1 - x[k] ) * asp[j] ) * ( sim_fc_st_act_st[j] ) )

      projected_forecast_st[j] = round(projected_forecast_st[j])
      projected_forecast_SI[j] = round(projected_forecast_SI[j])
      project_closing_oh[j] = round(project_closing_oh[j])
      project_opening_oh[j] = round(project_opening_oh[j])
      project_st_act[j] = round(project_st_act[j])

      sum = sum + trade_spend[j]


      j = j + 1

      q = q + 1

    i = i + 1
    k = k + 2

    print("Budget Constraint SUM value after difference is used ?: ")
    print(budget - sum )


  return 1 if sum < budget else 0

In [ ]:
def constraint4(x):
  i = 0
  j=0
  GPSum = 0
  RevSum = 0
  projected_forecast_st = copy.deepcopy(sum_of_forecast_st)
  projected_forecast_SI = copy.deepcopy(sum_of_forecast_SI)
  project_closing_oh = copy.deepcopy(sum_of_closing_oh)
  project_opening_oh = copy.deepcopy(sum_of_opening_oh)
  project_st_act = copy.deepcopy(sum_of_forecast_st)
  project_st_act_minor = copy.deepcopy(sum_of_forecast_st)

  trade_spend = copy.deepcopy(sum_of_forecast_st) #trade spend
  trade_spend_minor = copy.deepcopy(sum_of_forecast_st)

  forecasted_st_ac = copy.deepcopy(sum_of_actual_st) #

  GP_base = copy.deepcopy(sum_of_actual_st) #

  sim_fc_st_act_st = copy.deepcopy(sum_of_actual_st) #

  GP = copy.deepcopy(sum_of_actual_st)

  rev_base = copy.deepcopy(sum_of_actual_st)

  rev = copy.deepcopy(sum_of_actual_st)

  k = 0

  q = 0
  while i < total_products:
    #j = 0
    q = 0
    while q < total_run:
      if (date_arr[j] < today_arr[j]):
        projected_forecast_st[j] = sum_of_forecast_st[j]
        projected_forecast_SI[j] = sum_of_forecast_SI[j]
        project_closing_oh[j] = sum_of_closing_oh[j]
        project_opening_oh[j] = sum_of_opening_oh[j]
        project_st_act[j] = sum_of_forecast_st[j]

        trade_spend[j] = 0

        forecasted_st_ac[j] = sum_of_actual_st[j]

        sim_fc_st_act_st[j] = sum_of_actual_st[j]
        
      else:
        projected_forecast_st[j] = sum_of_forecast_st[j] * (1 + x[k]) * ped[j]
        projected_forecast_SI[j] = sum_of_forecast_SI[j] * (1 + x[k+1])
        if (j>0):
          if ((project_closing_oh[j-1] + projected_forecast_SI[j] - projected_forecast_st[j])>0):
            project_closing_oh[j] = project_closing_oh[j-1] + projected_forecast_SI[j] - projected_forecast_st[j]
          else:
            project_closing_oh[j] = 0
        else:
          project_closing_oh[j] = sum_of_closing_oh[j]
        if (j>0):
          project_opening_oh[j] = project_closing_oh[j-1]
        else:
          project_opening_oh[j] = sum_of_opening_oh[j]

        project_st_act_minor[j] = project_opening_oh[j] + projected_forecast_SI[j]
        project_st_act[j] = min(project_st_act_minor[j],projected_forecast_st[j])

        trade_spend_minor[j] = project_st_act[j] * asp[j] * x[k]
        trade_spend[j] = round(trade_spend_minor[j],-1)


        forecasted_st_ac[j] = min((sum_of_forecast_SI[j] + sum_of_opening_oh[j]), sum_of_forecast_st[j])

        sim_fc_st_act_st[j] = round(min((projected_forecast_SI[j] + project_opening_oh[j]), projected_forecast_st[j]))


      GP_base[j] = round( ( asp[j] - cost[j] ) * ( forecasted_st_ac[j]) )

      if (date_arr[j] < today_arr[j]):
        GP[j] = GP_base[j]
      else: 
        GP[j] = round( (( asp[j] * (1 - x[k]) - cost[j] ) * ( sim_fc_st_act_st[j] )), - 1)


      rev_base[j] = round( ( asp[j] * forecasted_st_ac[j] ) )


      if (date_arr[j] < today_arr[j]):
        rev[j] = rev_base[j]
      else: 
        rev[j] = round( ( ( 1 - x[k] ) * asp[j] ) * ( sim_fc_st_act_st[j] ) )

      projected_forecast_st[j] = round(projected_forecast_st[j])
      projected_forecast_SI[j] = round(projected_forecast_SI[j])
      project_closing_oh[j] = round(project_closing_oh[j])
      project_opening_oh[j] = round(project_opening_oh[j])
      project_st_act[j] = round(project_st_act[j])

      GPSum = GPSum + GP[j]
      RevSum = RevSum + rev[j]


      j = j + 1

      q = q + 1

    i = i + 1
    k = k + 2



  TotalSum = GPSum/RevSum

  return 1 if TotalSum > gp_min else 0
  #return TotalSum - gp_min


In [ ]:
def constraint5(x):
  return x[0]

def constraint6(x):
  return x[1]

def constraint7(x):
  return x[2]

def constraint8(x):
  return x[3]



def constraint9(x):
  return x[4]

def constraint10(x):
  return x[5]

def constraint11(x):
  return x[6]

def constraint12(x):
  return x[7]


def constraint13(x):
  return x[8]

def constraint14(x):
  return x[9]

def constraint15(x):
  return x[10]

def constraint16(x):
  return x[11]


def constraint17(x):
  return x[12]

def constraint18(x):
  return x[13]

def constraint19(x):
  return x[14]

def constraint20(x):
  return x[15]



def constraint21(x):
  return x[16]

def constraint22(x):
  return x[17]

def constraint23(x):
  return x[18]

def constraint24(x):
  return x[19]

In [ ]:
con1 = {'type': 'ineq', 'fun': constraint1}
con2 = {'type': 'ineq', 'fun': constraint2}
con3 = {'type': 'ineq', 'fun': constraint3}
con4 = {'type': 'ineq', 'fun': constraint4}
con5 = {'type': 'ineq', 'fun': constraint5}
con6 = {'type': 'ineq', 'fun': constraint6}






cons = ([con1, con2, con3, con4,con5,con6])

In [ ]:
import copy 


In [ ]:
def objective(x, sign=-1):
  i = 0
  j=0
  sum = 0
  tradespend_sum = 0
  GPSum = 0 
  RevSum = 0
  GP_percent_sum = 0
  projected_forecast_st = copy.deepcopy(sum_of_forecast_st)
  projected_forecast_SI = copy.deepcopy(sum_of_forecast_SI)
  project_closing_oh = copy.deepcopy(sum_of_closing_oh)
  project_opening_oh = copy.deepcopy(sum_of_opening_oh)
  project_st_act = copy.deepcopy(sum_of_forecast_st)
  project_st_act_minor = copy.deepcopy(sum_of_forecast_st)

  trade_spend = copy.deepcopy(sum_of_forecast_st) #trade spend
  trade_spend_minor = copy.deepcopy(sum_of_forecast_st)

  forecasted_st_ac = copy.deepcopy(sum_of_actual_st) #

  GP_base = copy.deepcopy(sum_of_actual_st) #

  sim_fc_st_act_st = copy.deepcopy(sum_of_actual_st) #

  GP = copy.deepcopy(sum_of_actual_st)

  rev_base = copy.deepcopy(sum_of_actual_st)

  rev = copy.deepcopy(sum_of_actual_st)

  k = 0
  q = 0
  while i < total_products:
    q = 0
    while q < total_run: #iski waja se change kia tha 120 to 1200, there should be another index 
      if (date_arr[j] < today_arr[j]):
        projected_forecast_st[j] = sum_of_forecast_st[j]
        projected_forecast_SI[j] = sum_of_forecast_SI[j]
        project_closing_oh[j] = sum_of_closing_oh[j]
        project_opening_oh[j] = sum_of_opening_oh[j]
        project_st_act[j] = sum_of_forecast_st[j]

        trade_spend[j] = 0

        forecasted_st_ac[j] = sum_of_actual_st[j]

        sim_fc_st_act_st[j] = sum_of_actual_st[j]
        
      else:
        projected_forecast_st[j] = sum_of_forecast_st[j] * (1 + x[k]) * ped[j]
        projected_forecast_SI[j] = sum_of_forecast_SI[j] * (1 + x[k+1])
        if (j>0):
          if ((project_closing_oh[j-1] + projected_forecast_SI[j] - projected_forecast_st[j])>0):
            project_closing_oh[j] = project_closing_oh[j-1] + projected_forecast_SI[j] - projected_forecast_st[j]
          else:
            project_closing_oh[j] = 0
        else:
          project_closing_oh[j] = sum_of_closing_oh[j]
        if (j>0):
          project_opening_oh[j] = project_closing_oh[j-1]
        else:
          project_opening_oh[j] = sum_of_opening_oh[j]

        project_st_act_minor[j] = project_opening_oh[j] + projected_forecast_SI[j]
        project_st_act[j] = min(project_st_act_minor[j],projected_forecast_st[j])

        trade_spend_minor[j] = project_st_act[j] * asp[j] * x[k]
        trade_spend[j] = round(trade_spend_minor[j],-1)


        forecasted_st_ac[j] = min((sum_of_forecast_SI[j] + sum_of_opening_oh[j]), sum_of_forecast_st[j])

        sim_fc_st_act_st[j] = round(min((projected_forecast_SI[j] + project_opening_oh[j]), projected_forecast_st[j]))


      GP_base[j] = round( ( asp[j] - cost[j] ) * ( forecasted_st_ac[j]) )

      if (date_arr[j] < today_arr[j]):
        GP[j] = GP_base[j]
      else: 
        GP[j] = round( (( asp[j] * (1 - x[k]) - cost[j] ) * ( sim_fc_st_act_st[j] )), - 1)


      rev_base[j] = round( ( asp[j] * forecasted_st_ac[j] ) )


      if (date_arr[j] < today_arr[j]):
        rev[j] = rev_base[j]
      else: 
        rev[j] = round( ( ( 1 - x[k] ) * asp[j] ) * ( sim_fc_st_act_st[j] ) )

      projected_forecast_st[j] = round(projected_forecast_st[j])
      projected_forecast_SI[j] = round(projected_forecast_SI[j])
      project_closing_oh[j] = round(project_closing_oh[j])
      project_opening_oh[j] = round(project_opening_oh[j])
      project_st_act[j] = round(project_st_act[j])


      sum = sum + project_st_act[j]

      tradespend_sum = tradespend_sum + trade_spend[j]

      GPSum = GPSum + GP[j]
      RevSum = RevSum + rev[j]


      j = j + 1

      q = q + 1

    i = i + 1
    k = k + 2



  GP_percent_sum = GPSum/RevSum


  print(" SUM OF OBJECTIVE FUNCTION... ")
  print(sum)

  print("X IN OBJECTIVE FUNCTION... ... ")
  print(x)

  return sign*(sum)

In [ ]:
def objective2(x, sign=-1):
  global ff_index
  ff_index = 0
  i = 0
  j=0
  sum = 0
  tradespend_sum = 0
  GPSum = 0 
  RevSum = 0
  GP_percent_sum = 0
  projected_forecast_st = copy.deepcopy(sum_of_forecast_st)
  projected_forecast_SI = copy.deepcopy(sum_of_forecast_SI)
  project_closing_oh = copy.deepcopy(sum_of_closing_oh)
  project_opening_oh = copy.deepcopy(sum_of_opening_oh)
  project_st_act = copy.deepcopy(sum_of_forecast_st)
  project_st_act_minor = copy.deepcopy(sum_of_forecast_st)

  trade_spend = copy.deepcopy(sum_of_forecast_st) #trade spend
  trade_spend_minor = copy.deepcopy(sum_of_forecast_st)

  forecasted_st_ac = copy.deepcopy(sum_of_actual_st) #

  GP_base = copy.deepcopy(sum_of_actual_st) #

  sim_fc_st_act_st = copy.deepcopy(sum_of_actual_st) #

  GP = copy.deepcopy(sum_of_actual_st)

  rev_base = copy.deepcopy(sum_of_actual_st)

  rev = copy.deepcopy(sum_of_actual_st)

  k = 0

  q = 0
  while i < total_products:
    #j = 0
    q = 0
    while q < total_run:
      if (date_arr[j] < today_arr[j]):
        projected_forecast_st[j] = sum_of_forecast_st[j]
        projected_forecast_SI[j] = sum_of_forecast_SI[j]
        project_closing_oh[j] = sum_of_closing_oh[j]
        project_opening_oh[j] = sum_of_opening_oh[j]
        project_st_act[j] = sum_of_forecast_st[j]

        trade_spend[j] = 0

        forecasted_st_ac[j] = sum_of_actual_st[j]

        sim_fc_st_act_st[j] = sum_of_actual_st[j]
        
      else:
        projected_forecast_st[j] = sum_of_forecast_st[j] * (1 + x[k]) * ped[j]
        projected_forecast_SI[j] = sum_of_forecast_SI[j] * (1 + x[k+1])
        if (j>0):
          if ((project_closing_oh[j-1] + projected_forecast_SI[j] - projected_forecast_st[j])>0):
            project_closing_oh[j] = project_closing_oh[j-1] + projected_forecast_SI[j] - projected_forecast_st[j]
          else:
            project_closing_oh[j] = 0
        else:
          project_closing_oh[j] = sum_of_closing_oh[j]
        if (j>0):
          project_opening_oh[j] = project_closing_oh[j-1]
        else:
          project_opening_oh[j] = sum_of_opening_oh[j]

        project_st_act_minor[j] = project_opening_oh[j] + projected_forecast_SI[j]
        project_st_act[j] = min(project_st_act_minor[j],projected_forecast_st[j])

        trade_spend_minor[j] = project_st_act[j] * asp[j] * x[k]
        trade_spend[j] = round(trade_spend_minor[j],-1)


        forecasted_st_ac[j] = min((sum_of_forecast_SI[j] + sum_of_opening_oh[j]), sum_of_forecast_st[j])

        sim_fc_st_act_st[j] = round(min((projected_forecast_SI[j] + project_opening_oh[j]), projected_forecast_st[j]))


      GP_base[j] = round( ( asp[j] - cost[j] ) * ( forecasted_st_ac[j]) )

      if (date_arr[j] < today_arr[j]):
        GP[j] = GP_base[j]
      else: 
        GP[j] = round( (( asp[j] * (1 - x[k]) - cost[j] ) * ( sim_fc_st_act_st[j] )), - 1)


      rev_base[j] = round( ( asp[j] * forecasted_st_ac[j] ) )


      if (date_arr[j] < today_arr[j]):
        rev[j] = rev_base[j]
      else: 
        rev[j] = round( ( ( 1 - x[k] ) * asp[j] ) * ( sim_fc_st_act_st[j] ) )

      projected_forecast_st[j] = round(projected_forecast_st[j])
      projected_forecast_SI[j] = round(projected_forecast_SI[j])
      project_closing_oh[j] = round(project_closing_oh[j])
      project_opening_oh[j] = round(project_opening_oh[j])
      project_st_act[j] = round(project_st_act[j])


      sum = sum + project_st_act[j]

      tradespend_sum = tradespend_sum + trade_spend[j]

      GPSum = GPSum + GP[j]
      RevSum = RevSum + rev[j]


      j = j + 1
      q = q + 1

    i = i + 1
    k = k + 2



  GP_percent_sum = GPSum/RevSum


  final_frame['Projected Forecast ST']=pd.Series(projected_forecast_st)

  final_frame['Projected Forecast SI']=pd.Series(projected_forecast_SI)

  final_frame['Project Opening OH']=pd.Series(project_opening_oh)

  final_frame['Project Closing OH']=pd.Series(project_closing_oh)

  final_frame['Project ST (Act)']=pd.Series(project_st_act)

  final_frame['Trade Spend']=pd.Series(trade_spend)

  final_frame['Forecasted ST/AC']=pd.Series(forecasted_st_ac)

  final_frame['Cost']=pd.Series(cost)

  final_frame['GP']=pd.Series(GP)

  final_frame['Rev']=pd.Series(rev)

  final_frame['GP Base']=pd.Series(GP_base)

  final_frame['Rev Base']=pd.Series(rev_base)

  final_frame['Sim FC ST/Act ST']=pd.Series(sim_fc_st_act_st)


  print(" SUM OF OBJECTIVE FUNCTION... ")
  print(sum)

  print("X IN OBJECTIVE FUNCTION... ... ")
  print(x)

  print("projected_forecast_st IN OBJECTIVE FUNCTION 2... ... ")

  print(projected_forecast_st)
  print("projected_forecast_st IN OBJECTIVE FUNCTION 120... ... ")
  print(projected_forecast_st[120])
  print("projected_forecast_st IN OBJECTIVE FUNCTION 240... ... ")
  print(projected_forecast_st[240])

  return sign*(sum)

In [ ]:

#solution = minimize_scalar(objective,discount,method='bounded',\
#                    bounds=bnds,constraints=cons) #SLSQP

solution = minimize(objective,discount,method='COBYLA',\
                    bounds=bnds, constraints=cons) #SLSQP


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:528: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)


Streaming output truncated to the last 5000 lines.
-2284150.0
Budget Constraint SUM value after difference is used ?: 
-2711080.0
Budget Constraint SUM value after difference is used ?: 
-2888350.0
Budget Constraint SUM value after difference is used ?: 
-2665590.0
Budget Constraint SUM value after difference is used ?: 
-3675300.0
 SUM OF OBJECTIVE FUNCTION... 
132153
X IN OBJECTIVE FUNCTION... ... 
[ 0.1510709   0.4         0.01061145  0.4         0.1653212   0.4
  0.2         0.4         0.19043131  0.4        -0.05799928  0.4
  0.10851917  0.4         0.1741204   0.4        -0.03471546  0.4
  0.2         0.4       ]
DICSOUNT CONSTRAINT 1 ....
[0.15107090221253797, 0.010611453071994244, 0.16532120002572612, 0.2, 0.19043131092247245, -0.05799928328058115, 0.10851917094314603, 0.17412039690771203, -0.03471545513149573, 0.20000000000000004]
ChangeinSI CONSTRAINT 2 ....
[0.4, 0.4, 0.4000000000000002, 0.4, 0.4, 0.4, 0.4000000000000002, 0.4000000000000002, 0.4, 0.4]
Budget Constraint SUM 

In [ ]:
final_frame.drop(['today'], axis = 1, inplace= True)

In [ ]:
x = solution.x

In [ ]:
zero_check =0

while zero_check < total_products:
  if(x[zero_check]<0):
    x[zero_check] = 0
  
  zero_check = zero_check + 1


In [ ]:
t = 0
u = 0

while t < run_product:
  final_frame.loc[t,'% Discount'] = x[u]
  final_frame.loc[t,'% Change in SI'] = x[u+1]

  t = t + 1

  if (t%120==0):
    u = u + 2



In [ ]:

final_obj = str(-1*objective2(x))
# show final objective
print('Final Objective: ' + final_obj)

final_frame['Final Objective Sum'] = final_obj

 SUM OF OBJECTIVE FUNCTION... 
132168
X IN OBJECTIVE FUNCTION... ... 
[ 0.15136662  0.4         0.02368295  0.39973004  0.17210486  0.4
  0.19603164  0.39909548  0.19176559  0.39977694 -0.05617101  0.39944666
  0.09918147  0.39999883  0.13745578  0.4        -0.04602992  0.39994355
  0.17854946  0.39997414]
projected_forecast_st IN OBJECTIVE FUNCTION 2... ... 
[85, 103, 35, 108, 49, 55, 118, 77, 130, 160, 115, 106, 101, 125, 42, 124, 103, 78, 50, 144, 34, 115, 95, 96, 101, 160, 211, 34, 200, 130, 170, 73, 154, 224, 167, 94, 151, 244, 67, 188, 131, 251, 102, 174, 108, 175, 178, 244, 206, 60, 91, 53, 101, 60, 258, 65, 62, 128, 142, 98, 208, 115, 34, 42, 22, 43, 72, 60, 102, 72, 77, 47, 49, 151, 109, 72, 77, 31, 86, 100, 83, 122, 56, 134, 57, 46, 148, 69, 107, 49, 64, 96, 125, 22, 116, 34, 95, 81, 107, 69, 92, 77, 116, 100, 47, 75, 25, 31, 85, 95, 40, 138, 38, 124, 125, 81, 104, 31, 91, 44, 102, 124, 42, 130, 59, 66, 142, 92, 156, 192, 138, 127, 121, 150, 50, 149, 124, 94, 60, 173, 41, 138

In [ ]:
#final_frame.tail()

In [ ]:
#final_frame = final_frame[final_frame.Retailer.notnull()]

In [ ]:
final_frame.head()

1,Retailer,Item Code,Division Name,Business Unit,Item Description,Location Code,Location Name,City,State,Zip Code,Date,Week,Sum of Actual ST,Sum of Forecast ST,Sum of Forecast SI,Sum of Opening OH,Sum of Closing OH,PED,ASP,% Discount,% Change in SI,Projected Forecast ST,Projected Forecast SI,Project Opening OH,Project Closing OH,Project ST (Act),Trade Spend,Forecasted ST/AC,Cost,GP,Rev,GP Base,Rev Base,Sim FC ST/Act ST,Final Objective Sum
0,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-01,2020-11-01 00:00:00,74,85,0,0,459,1.13,2599.99,0.151367,0.4,85,0,0,459,85,0.0,74,1040,115439.0,192399,115439,192399,74,132168
1,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-02,2020-11-02 00:00:00,109,103,215,459,565,1.13,2599.99,0.151367,0.4,103,215,459,565,103,0.0,109,1040,170039.0,283399,170039,283399,109,132168
2,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-03,2020-11-02 00:00:00,31,35,0,565,534,1.13,2599.99,0.151367,0.4,35,0,565,534,35,0.0,31,1040,48360.0,80600,48360,80600,31,132168
3,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-04,2020-11-02 00:00:00,110,108,0,534,424,1.13,2599.99,0.151367,0.4,108,0,534,424,108,0.0,110,1040,171599.0,285999,171599,285999,110,132168
4,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-05,2020-11-02 00:00:00,46,49,143,424,521,1.13,2599.99,0.151367,0.4,49,143,424,521,49,0.0,46,1040,71760.0,119600,71760,119600,46,132168


In [ ]:
final_frame.tail()

1,Retailer,Item Code,Division Name,Business Unit,Item Description,Location Code,Location Name,City,State,Zip Code,Date,Week,Sum of Actual ST,Sum of Forecast ST,Sum of Forecast SI,Sum of Opening OH,Sum of Closing OH,PED,ASP,% Discount,% Change in SI,Projected Forecast ST,Projected Forecast SI,Project Opening OH,Project Closing OH,Project ST (Act),Trade Spend,Forecasted ST/AC,Cost,GP,Rev,GP Base,Rev Base,Sim FC ST/Act ST,Final Objective Sum
1195,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-24,2021-02-22 00:00:00,0,60,0,0,0,0.95,999.99,0.178549,0.399974,67,0,107,40,67,11990.0,0,600,14840.0,55037,0,0,67,132168
1196,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-25,2021-02-22 00:00:00,0,79,118,0,39,0.95,999.99,0.178549,0.399974,88,165,40,117,88,15790.0,79,600,19490.0,72287,31599,78999,88,132168
1197,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-26,2021-02-22 00:00:00,0,64,0,39,0,0.95,999.99,0.178549,0.399974,72,0,117,45,72,12790.0,39,600,15940.0,59144,15600,39000,72,132168
1198,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-27,2021-02-22 00:00:00,0,66,0,0,0,0.95,999.99,0.178549,0.399974,74,0,45,0,45,8030.0,0,600,9960.0,36965,0,0,45,132168
1199,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-28,2021-02-22 00:00:00,0,42,0,0,0,0.95,999.99,0.178549,0.399974,47,0,0,0,0,0.0,0,600,0.0,0,0,0,0,132168


In [ ]:
# print solution
print('Solution')
print('x = ' + str(x))

Solution
x = [ 0.15136662  0.4         0.02368295  0.39973004  0.17210486  0.4
  0.19603164  0.39909548  0.19176559  0.39977694 -0.05617101  0.39944666
  0.09918147  0.39999883  0.13745578  0.4        -0.04602992  0.39994355
  0.17854946  0.39997414]


In [ ]:
final_frame['Max SI'] =   max_si 

final_frame['Max Discount'] =   max_discount

final_frame['GP Min'] =   gp_min

final_frame['Budget'] =   budget 

final_frame['Scenario'] =   scenario 

In [ ]:
final_frame.head()

1,Retailer,Item Code,Division Name,Business Unit,Item Description,Location Code,Location Name,City,State,Zip Code,Date,Week,Sum of Actual ST,Sum of Forecast ST,Sum of Forecast SI,Sum of Opening OH,Sum of Closing OH,PED,ASP,% Discount,% Change in SI,Projected Forecast ST,Projected Forecast SI,Project Opening OH,Project Closing OH,Project ST (Act),Trade Spend,Forecasted ST/AC,Cost,GP,Rev,GP Base,Rev Base,Sim FC ST/Act ST,Final Objective Sum,Max SI,Max Discount,GP Min,Budget,Scenario
0,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-01,2020-11-01 00:00:00,74,85,0,0,459,1.13,2599.99,0.151367,0.4,85,0,0,459,85,0.0,74,1040,115439.0,192399,115439,192399,74,132168,0.4,0.2,0.5,600000,304
1,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-02,2020-11-02 00:00:00,109,103,215,459,565,1.13,2599.99,0.151367,0.4,103,215,459,565,103,0.0,109,1040,170039.0,283399,170039,283399,109,132168,0.4,0.2,0.5,600000,304
2,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-03,2020-11-02 00:00:00,31,35,0,565,534,1.13,2599.99,0.151367,0.4,35,0,565,534,35,0.0,31,1040,48360.0,80600,48360,80600,31,132168,0.4,0.2,0.5,600000,304
3,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-04,2020-11-02 00:00:00,110,108,0,534,424,1.13,2599.99,0.151367,0.4,108,0,534,424,108,0.0,110,1040,171599.0,285999,171599,285999,110,132168,0.4,0.2,0.5,600000,304
4,Best Buy,surf001,Surface Devices,Surface Laptop,Surface Laptop i7/8/256,BBY000835,86TH AND LEXINGTN NY,New York,New York,10028-2105,2020-11-05,2020-11-02 00:00:00,46,49,143,424,521,1.13,2599.99,0.151367,0.4,49,143,424,521,49,0.0,46,1040,71760.0,119600,71760,119600,46,132168,0.4,0.2,0.5,600000,304


In [ ]:
final_frame.tail()

1,Retailer,Item Code,Division Name,Business Unit,Item Description,Location Code,Location Name,City,State,Zip Code,Date,Week,Sum of Actual ST,Sum of Forecast ST,Sum of Forecast SI,Sum of Opening OH,Sum of Closing OH,PED,ASP,% Discount,% Change in SI,Projected Forecast ST,Projected Forecast SI,Project Opening OH,Project Closing OH,Project ST (Act),Trade Spend,Forecasted ST/AC,Cost,GP,Rev,GP Base,Rev Base,Sim FC ST/Act ST,Final Objective Sum,Max SI,Max Discount,GP Min,Budget,Scenario
1195,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-24,2021-02-22 00:00:00,0,60,0,0,0,0.95,999.99,0.178549,0.399974,67,0,107,40,67,11990.0,0,600,14840.0,55037,0,0,67,132168,0.4,0.2,0.5,600000,304
1196,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-25,2021-02-22 00:00:00,0,79,118,0,39,0.95,999.99,0.178549,0.399974,88,165,40,117,88,15790.0,79,600,19490.0,72287,31599,78999,88,132168,0.4,0.2,0.5,600000,304
1197,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-26,2021-02-22 00:00:00,0,64,0,39,0,0.95,999.99,0.178549,0.399974,72,0,117,45,72,12790.0,39,600,15940.0,59144,15600,39000,72,132168,0.4,0.2,0.5,600000,304
1198,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-27,2021-02-22 00:00:00,0,66,0,0,0,0.95,999.99,0.178549,0.399974,74,0,45,0,45,8030.0,0,600,9960.0,36965,0,0,45,132168,0.4,0.2,0.5,600000,304
1199,Walmart,surf005,Surface Devices,Surface Pro X,Surface ProX E/8/128 LTE,WLY000609,BROADWAY NOHO NY,New York,New York,10012-2600,2021-02-28,2021-02-22 00:00:00,0,42,0,0,0,0.95,999.99,0.178549,0.399974,47,0,0,0,0,0.0,0,600,0.0,0,0,0,0,132168,0.4,0.2,0.5,600000,304


In [ ]:
!pip install xlsxwriter
import xlsxwriter
writer = pd.ExcelWriter("model_scenarios.xlsx", engine = 'xlsxwriter')
#writer = pd.ExcelWriter("model_scenarios.xlsx", engine = 'openpyxl', mode='a')
final_frame.to_excel(writer, sheet_name = "scenario_"+str(scenario))
writer.save()
writer.close()

     |████████████████████████████████| 153kB 27.3MB/s 


/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
